##  Optimze and a Dense Neural Network for gap filling and feature identification

** With a few tweaks to RepRunner, an LSTM can be run instead

In [94]:
import time
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from itertools import combinations
from functools import partial
from multiprocessing import Pool
from sklearn import metrics
from sklearn.model_selection import train_test_split

## Personal Modules
import ReadStandardTimeFill as RSTF
import importlib
import DenseNet as Dense
import MiscFuncs as MF
importlib.reload(Dense)
importlib.reload(RSTF)
importlib.reload(MF)

# %matplotlib inline

%matplotlib notebook
%config IPCompleter.greedy=True

from scipy.optimize import minimize, curve_fit
from scipy.stats import norm
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern, WhiteKernel, ConstantKernel
from matplotlib import cm

from scipy import stats
from statsmodels.stats.multicomp import pairwise_tukeyhsd

from ipywidgets import FloatProgress
from IPython.display import display
import os  
try:pool.close()
except:pass

# Model

In [ ]:

MP=False
Scope = 'Test'
cwd = os.getcwd()
# for Site in ['Illisarvik','FishIsland']:
Site='Illisarvik'
target='fco2'
Inputs=['PPFD_Avg']#,'VWC','Temp','wind_speed']
#     for FillVar in ['fco2','fch4']:
# Runs,params = Dense.Params(Scope,target,MP)
params = Dense.Params(Scope,target,MP)

params['Dpath'] = cwd+'/'+Site+'/'
params['Spath'] = params['Dpath']+'/TempWeights/'
params['Sname'] = 'Test'
params['Inputs'] = Inputs
params['Model'] = '+'.join(params['Inputs'])

# params['Loss']='mean_absolute_error'

if __name__ == '__main__':
    if params['proc']>1:
        pool = Pool(processes=3,maxtasksperchild=75)
    else:
        pool = None
    
    RST = RSTF.ReadStandardTimeFill(params['Dpath']+'ECData.csv',resample='1H')
    RST.Scale(params['target'],params['Inputs'])
    y = RST.y*1.0
    X = RST.X*1.0
    
    params['N']=int(y.shape[0]/30)
    print(y.shape,params['N'])
    
#     Y_hat_train,Y_hat_val,y_true,X_true,count_train,count_val = 
    Dense.RunNN(params,X,y,RST.YScaled,RST.XScaled,pool)
#     Y_hat,y_true = np.squeeze(Y_hat),np.squeeze(y_true)
    
    

    if pool is not None:
        pool.close()
      
    
print('Done!!')

(500,) 16
0.9500000000000001
Saved model to disk


# Sorting

# CI and PI

# Map Results

In [ ]:
# X = RST.X#Scaled.transform(X_true)
# print(X.shape)
# params['Sname']='Test'
# params['Loss']='mean_absolute_error'
# Y_fill = []
# MSE = []
# for i in range(params['K']):
#     params['iteration']=i
#     Empty_Mod = Dense.Load_Model(params)
#     Model = Dense.Load_Weights(Empty_Mod,params) 
#     Y = RST.YScaled.inverse_transform(Model.predict(X).reshape(-1,1))
#     mse = (metrics.mean_squared_error(RST.y,Y))
#     Y_fill.append(Y)
#     MSE.append(mse)
# Y_fill = np.asanyarray(Y_fill).mean(axis=-1)
# Y_fill_bar = Y_fill.mean(axis=0)
# MSE = np.asanyarray(MSE)
# CI = stats.t.ppf(1-0.025,i)*MSE.std()/(i)**.5
# print(CI)

# params['Sname']='Var'
# params['iteration']=1
# params['Loss']='Boot_Loss'
# Empty_Mod = Dense.Load_Model(params)
# Model = Dense.Load_Weights(Empty_Mod,params) 
# YVar=YScaled.inverse_transform(Model.predict(X).reshape(-1,1))
# X_back = np.squeeze(RST.XScaled.inverse_transform(X))

# # plt.figure()
# # plt.plot(Yfill)
# Data = pd.DataFrame(data=X_back,columns=params['Inputs'])
# Data[target] = np.squeeze(Y_fill_bar)
# Data['True'] = np.squeeze(RST.y)
# # Data = pd.DataFrame({'PPFD':X_back,Target:np.squeeze(Yfill),'Var':np.squeeze(YVar)})
# Data['SE'] = 1/(params['K']-1)*((Y_fill-Y_fill_bar)**2).sum(axis=0)
# Data['Var'] = np.squeeze(YVar)
# Data['CI']=stats.t.ppf(1-0.025,params['K'])*(Data['SE'])**.5
# Data['PI']=stats.t.ppf(1-0.025,params['K'])*((Data['Var']+Data['SE'])**.5) #the accuracy of our estimate with respect to the observed output
# # Map_Val='VWC'


plt.figure(figsize=(8,7))
Data = Data.sort_values(by='PPFD_Avg')
Data.index = Data.PPFD_Avg


plt.scatter(Data.index,Data['True'],edgecolor='black',facecolor='white')
plt.plot(Data.index,Data[target],label=
        params['target']+' Model\nRMSE: '+str(np.round(metrics.mean_squared_error(Data['True'],
                                                                   Data[params['target']])**2,3)))
# plt.plot(Data.index,Data['Var'],label=
#         params['target']+' Model\nRMSE: '+str(np.round(metrics.mean_squared_error(Data['True'],
#                                                                    Data[params['target']])**2,3)))


plt.fill_between(Data.index, Data[target]-Data['PI'], 
                 Data[target]+Data['PI'],  color = 'green', alpha = 0.4, 
                 label = '95% PI')
plt.fill_between(Data.index, Data[target]-Data['CI'], 
                 Data[target]+Data['CI'],  color = 'red', alpha = 0.4, 
                 label = '95% CI')
plt.legend()

# The "Optimum" Sized Model

In [ ]:
# pool.close()
# Site = 'Illisarvik'#'FishIsland'#
Scope = 'Test'
cwd = os.getcwd()
def ModSelect(Scope,Site):
    if Site == 'Illisarvik':
#         BaseFactors = ['Sedge','Shrub','Grass','Sparse','Out_of_Basin']
#         BaseFactors = []
        if Scope == 'Full':
            Model = ['H','wind_speed','air_pressure','PPFD_Avg','AirTC_Avg','VPD',
                    'Temp','VWC','Sedge','Shrub','Grass','Sparse','Out_of_Basin']
        if Scope == 'Test':
            Model = ['PPFD_Avg','wind_speed']
    if Site == 'FishIsland':
        BaseFactors = []
        if Scope == 'Full':
            Model = ['H','Wind Spd','air pressure','Ta','Rn','PPFD','Rain','Water Table',
            'Ts 2.5 cm','Ts 15 cm','VWC','Active Layer','24H Rain','Wtr Tbl Trnd']
        if Scope == 'Test':
            Model = ['H','Water Table','Wind Spd','Active Layer']
    return(Model)

# def Params(Func,Y,MP = True):
#     params = {}
#     params['proc']=3
#     if MP == False:
#         params['proc']=1
#     if Func == 'Full':
#         epochs = 200
#         K = 30
#         splits_per_mod = 1
#         N = np.linspace(200,20,10,dtype='int32')
#     elif Func == 'Test':
#         epochs = 200
#         K = 30
#         splits_per_mod = 1
#         N = np.linspace(70,10,5,dtype='int32')
#     N = np.repeat(N,K)
#     d = {'N':N.astype(int)}
#     Runs = pd.DataFrame(data=d)
#     Runs['MAE'] = 0.0
#     Runs['R2'] = 0.0
#     Runs['Model']=0
#     params['K'] = K
#     params['epochs'] = epochs
#     params['Y'] = Y
#     params['splits_per_mod'] = splits_per_mod
#     params['Save'] = {}
#     params['Save']['Weights']=False
#     params['Save']['Model']=False
    
#     return(Runs,params)


# MP=False

# if Scope == 'Full':
#     MP = True
# if __name__=='__main__'and MP==True:
#     pool = Pool(processes=3,maxtasksperchild=75)
# else:pool=None
    
# # for Site in ['Illisarvik','FishIsland']:
# Site='Illisarvik'
# FillVar = 'fco2'
# #     for FillVar in ['fco2','fch4']:
# Runs,params = MF.Params(Scope,FillVar,MP)
# FullModel = ModSelect(Scope,Site)
# print(FullModel)
# params['Dpath'] = cwd+'/'+Site+'/'
# params['Prelim_N']=True
# Best,Scores,ModelRuns = MF.FactorTest(params,FullModel,Runs)
# print(Best,Scores)
# Scores,ModelRuns = Best_Fill(Best,Runs,Scores,params)
# Scores.to_csv(params['Dpath']+FillVar+'/GapFillingSummary.csv')
# ModelRuns.to_csv(params['Dpath']+FillVar+'/GapFilled.csv')

# if __name__=='__main__'and MP==True:
#     pool.close()

In [ ]:
# Grp = Scores.groupby('Model').mean()
# Grp['SE'] = Scores[['Model','MAE']].groupby('Model').sem()
# # Grp['SE'] = Scores[['Key','MAE']].groupby('Key').sem()
# print(Grp)
# # plt.bar(Grp.index,Grp['MAE'],yerr=Grp['SE'])

In [ ]:
print('kitty')